# Grupo 7
Rubén Gómez Blanco,
Adrián Sanjuan Espejo 

In [1]:
import numpy as np
from pandas.io.parsers import read_csv
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.datasets import make_blobs
from scipy import integrate
from sklearn.preprocessing import PolynomialFeatures
from numpy import arange, meshgrid, sqrt
from google.colab import drive
from sklearn.model_selection import train_test_split
drive.mount('/content/drive')
from xgboost import XGBClassifier, plot_tree
from sklearn.model_selection import GridSearchCV
import time

Mounted at /content/drive


Cargamos los datos de la práctica 8

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

# Cargamos el dataset del iris
iris = load_iris()
iris.keys()

df = pd.DataFrame(data=iris['data'], columns=iris['feature_names']) 
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
sepal length (cm),150.0,5.843333,0.828066,4.3,5.1,5.80,6.4,7.9
sepal width (cm),150.0,3.057333,0.435866,2.0,2.8,3.00,3.3,4.4
petal length (cm),150.0,3.758000,1.765298,1.0,1.6,4.35,5.1,6.9
petal width (cm),150.0,1.199333,0.762238,0.1,0.3,1.30,1.8,2.5


Separamos los datos en entrenamiento, validación y test (60-20-20)

In [18]:
X=iris['data']
y=iris['target']

X_train_val, X_test, y_train_val, y_test = train_test_split(X,y, train_size=0.8, stratify=y, random_state = 10)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_val, y_train_val, test_size=0.25, stratify=y_train_val, random_state = 10)
len(X_train)

90

# MODELO

Exploramos distintos valores de los siguientes parámetros a ajustar en validación.

*  learning_rate: indica el ratio de aprendizaje en cada paso, utilizado para evitar el overfitting.
*  max_depth: máxima profundidad de un árbol.
*  n_estimators: número de árboles construidos por el modelo.
*  subsample: proporción de submuestras de las instancias de entrenamiento.
*  colsample_bytree: proporción de submuestras de columnas al construir cada árbol.

Se han considerado estos parámetros (y sus valores) porque hemos considerado que más afectarían al rendimiento del modelo (over y underfitting) para el problema que se está considerando.

In [32]:

'''
param_grid = {
    'learning_rate': [0.2, 0.1, 0.01, 0.001],
    'max_depth': [None, 3, 5, 7, 10],
    'n_estimators': [25, 50, 100, 200, 500],
    'subsample': [0.3, 0.5, 0.7, 0.9],
    'colsample_bytree': [0.5,0.75, 1]
}
'''

#Condiciones más restrictivas
param_grid = {
    'learning_rate': [0.2, 0.1, 0.01, 0.001],
    'max_depth': [1,2,3],
    'n_estimators': [10,25,50],
    'subsample': [0.3, 0.5, 0.7, 0.9],
    'colsample_bytree': [0.5,0.75, 1]
}
results = []

for lr in param_grid['learning_rate']:
  for max_d in param_grid['max_depth']:
    for n_est in param_grid['n_estimators']:
      for subs in param_grid['subsample']:
        for cols in param_grid['colsample_bytree']:
          model = XGBClassifier(learning_rate = lr,max_depth = max_d,  n_estimators = n_est, subsample = subs, colsample_bytree = cols, random_state = 10)
          
          start_time = time.time()
          model.fit(X_train, y_train)
          elapsed_time = round(time.time() - start_time, 2)
          
          y_pred = model.predict(X_train)
          acc_train = round(np.sum(y_pred == y_train) / len(y_train), 3)
          y_pred = model.predict(X_valid)
          acc_val = round(np.sum(y_pred == y_valid) / len(y_valid), 3)
          
          ele = (acc_train,acc_val,(lr,max_d,n_est,subs,cols), elapsed_time)
          results.append(ele)

          print(f'{(lr,max_d,n_est,subs,cols)}: Train accuracy {acc_train} / Validation accuracy {acc_val} / Time {elapsed_time}')
          


(0.2, 1, 10, 0.3, 0.5): Train accuracy 0.944 / Validation accuracy 1.0 / Time 0.02
(0.2, 1, 10, 0.3, 0.75): Train accuracy 0.933 / Validation accuracy 1.0 / Time 0.02
(0.2, 1, 10, 0.3, 1): Train accuracy 0.933 / Validation accuracy 1.0 / Time 0.15
(0.2, 1, 10, 0.5, 0.5): Train accuracy 0.933 / Validation accuracy 1.0 / Time 0.02
(0.2, 1, 10, 0.5, 0.75): Train accuracy 0.944 / Validation accuracy 0.967 / Time 0.02
(0.2, 1, 10, 0.5, 1): Train accuracy 0.956 / Validation accuracy 1.0 / Time 0.01
(0.2, 1, 10, 0.7, 0.5): Train accuracy 0.933 / Validation accuracy 0.967 / Time 0.01
(0.2, 1, 10, 0.7, 0.75): Train accuracy 0.933 / Validation accuracy 0.967 / Time 0.01
(0.2, 1, 10, 0.7, 1): Train accuracy 0.933 / Validation accuracy 0.967 / Time 0.01
(0.2, 1, 10, 0.9, 0.5): Train accuracy 0.933 / Validation accuracy 0.967 / Time 0.01
(0.2, 1, 10, 0.9, 0.75): Train accuracy 0.933 / Validation accuracy 0.967 / Time 0.01
(0.2, 1, 10, 0.9, 1): Train accuracy 0.933 / Validation accuracy 0.967 / Time

In [33]:
#Lista de mejores parametros
models_list = sorted(results, reverse=True, key=lambda x: x[1])
print(param_grid.keys())
print('\n')
for element in models_list[:15]:
  print(f"Model {element[2]} (time {element[3]}):")
  print(f"\Train accuracy: {element[0]}")
  print(f"\Validation accuracy: {element[1]}")
  print("------------")

dict_keys(['learning_rate', 'max_depth', 'n_estimators', 'subsample', 'colsample_bytree'])


Model (0.2, 1, 10, 0.3, 0.5) (time 0.02):
\Train accuracy: 0.944
\Validation accuracy: 1.0
------------
Model (0.2, 1, 10, 0.3, 0.75) (time 0.02):
\Train accuracy: 0.933
\Validation accuracy: 1.0
------------
Model (0.2, 1, 10, 0.3, 1) (time 0.15):
\Train accuracy: 0.933
\Validation accuracy: 1.0
------------
Model (0.2, 1, 10, 0.5, 0.5) (time 0.02):
\Train accuracy: 0.933
\Validation accuracy: 1.0
------------
Model (0.2, 1, 10, 0.5, 1) (time 0.01):
\Train accuracy: 0.956
\Validation accuracy: 1.0
------------
Model (0.2, 1, 25, 0.3, 0.5) (time 0.01):
\Train accuracy: 0.944
\Validation accuracy: 1.0
------------
Model (0.2, 1, 25, 0.3, 0.75) (time 0.01):
\Train accuracy: 0.944
\Validation accuracy: 1.0
------------
Model (0.2, 1, 25, 0.3, 1) (time 0.01):
\Train accuracy: 0.944
\Validation accuracy: 1.0
------------
Model (0.2, 1, 25, 0.5, 0.5) (time 0.01):
\Train accuracy: 0.933
\Validation ac

In [34]:
#Lista de peores parametros
models_list = sorted(results, reverse=False, key=lambda x: x[1])
print(param_grid.keys())
print('\n')
for element in models_list[:15]:
  print(f"Model {element[2]} (time {element[3]}):")
  print(f"\Train accuracy: {element[0]}")
  print(f"\Validation accuracy: {element[1]}")
  print("------------")

dict_keys(['learning_rate', 'max_depth', 'n_estimators', 'subsample', 'colsample_bytree'])


Model (0.2, 1, 10, 0.5, 0.75) (time 0.02):
\Train accuracy: 0.944
\Validation accuracy: 0.967
------------
Model (0.2, 1, 10, 0.7, 0.5) (time 0.01):
\Train accuracy: 0.933
\Validation accuracy: 0.967
------------
Model (0.2, 1, 10, 0.7, 0.75) (time 0.01):
\Train accuracy: 0.933
\Validation accuracy: 0.967
------------
Model (0.2, 1, 10, 0.7, 1) (time 0.01):
\Train accuracy: 0.933
\Validation accuracy: 0.967
------------
Model (0.2, 1, 10, 0.9, 0.5) (time 0.01):
\Train accuracy: 0.933
\Validation accuracy: 0.967
------------
Model (0.2, 1, 10, 0.9, 0.75) (time 0.01):
\Train accuracy: 0.933
\Validation accuracy: 0.967
------------
Model (0.2, 1, 10, 0.9, 1) (time 0.01):
\Train accuracy: 0.933
\Validation accuracy: 0.967
------------
Model (0.2, 1, 25, 0.9, 0.5) (time 0.01):
\Train accuracy: 0.944
\Validation accuracy: 0.967
------------
Model (0.2, 1, 50, 0.5, 0.75) (time 0.02):
\Train accuracy: 

In [38]:
lr,max_d,n_est,subs,cols = 0.2, 1, 10, 0.3, 0.75
model = XGBClassifier(learning_rate = lr,max_depth = max_d,  n_estimators = n_est, subsample = subs, colsample_bytree = cols, random_state = 100)
model.fit(X_train_val, y_train_val, verbose = 0)
y_pred_test = model.predict(X_test)

In [39]:
acc_test_def = np.sum(y_pred_test == y_test) / len(y_test)

In [40]:
print(f"Model {(lr,max_d,n_est,subs,cols)}:")
print(f"\Test accuracy: {acc_test_def}")
print("------------")

Model (0.2, 1, 10, 0.3, 0.75):
\Test accuracy: 1.0
------------


Los resultados obtenidos pueden deberse al escaso número de ejemplos en el dataset y la relativa simpleza del problema. Por esta razón, los resultados tanto en validación como en test son muy buenos independientemente de los valores de los parámetros utilizados. 
En el caso de tener un problema más complejo, los parámetros afectarían de la siguiente manera:

*  learning_rate: un learning rate bajo hace que el modelo aprenda más despacio, ajustandose en menor medida a los datos de entrenamiento, evitando así el overfitting.
*  max_depth: cuanto más profundo es el árbol, más se produce overfitting en el modelo. Esto se debe a que se crearán hojas con pocos ejemplos pero muy ajustados a ellos, por lo que no se generaliza.
*  n_estimators: cuanto más árboles se usen en el modelo, se producirá más overfitting al ajustarse demasiado a los datos de entrenamiento. Además aumentará considerablemente el tiempo de entrenamiento.
*  subsample: cuanto mayor sea esta proporción, más sobreaprende el modelo, ya que usrá cada vez más instancias de los datos de entrenamiento.
*  colsample_bytree: cuantas más columnas sean tomadas en cuenta en el modelo, mayor será el ajuste de este a los datos de entrenamiento.

El objetivo es encontrar los valores de los parámetros que eviten el overfitting pero que produzcan buenos resultados en el conjunto de validación, comprobando así que no se ha generalizado demasiado.